### Prototype Code : Prompt Generator

In [ ]:
# Prompt Generator + LLM

# CONTEXT
template = """
You are a quality assurance expert that generates functional test cases for websites. You take in a UI element and you generate a functional test case.

Here is the UI element (some elements have a link attached to them): {question}
ONLY output in the following format: 
"Objective"~"Preconditions"~"Test Steps"~"Expected Result"

DO NOT output any other text. DO NOT output 'Here are the test cases...', your output should be like the example output below.

Example Input:
Link Element: Home with URL : https://bicol-u.edu.ph/
Link Element: Academics with URL : https://bicol-u.edu.ph/#
...

Example Output:
"Verify the functionality of the Link Element 'Home'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Home\'' '3. Verify if the webpage opens in a new tab/window.'"~"Webpage 'https://bicol-u.edu.ph/' should open in a new tab/window."
"Verify the functionality of the Link Element 'Academics'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Academics\'' '3. Verify if the link url changes to \'https://bicol-u.edu.ph/#\'' '4. Verify if a dropdown below \'Academics\' is visible'"~"A dropdown should show below 'Academics', but the webpage does not change"
...

"""

# Load Model Chain
def load_model_chain(template : str =  template, model_str : str = "llama3.1"):
    # Prompt
    prompt = ChatPromptTemplate.from_template(template)
    # Model
    model = OllamaLLM(model=model_str)
    # Chain
    chain = prompt | model
    return chain

# Create Test Case Data
def create_test_cases(data, chain, model_str : str = "llama3.1" , template : str = template, batch_size : int = 10):
    
    return_data = []
    
    for sub_data in data:
        element_test_cases = []
        i = 0
        j = 0
        print(f"Batch Number: {ceil(len(sub_data)/batch_size)}")
        while (j<ceil(len(sub_data)/batch_size)):
            print(f"[{j}] Batch {str(len(sub_data[i:i+batch_size]))}")
            appending = []
            for dat in sub_data[i:i+batch_size]:
                appending.append(chain.invoke({"question": str(dat)}))
            element_test_cases.append(appending)
            i+=batch_size
            j+=1
            prompt = ChatPromptTemplate.from_template(template)
            model = OllamaLLM(model=model_str)
            chain = prompt | model
        return_data.append(element_test_cases)
    return return_data


### Version 1 : Dissection and Simplification

In [ ]:
# Langchain and Ollama
import langchain
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

# CONTEXT V1
template = """
You are a quality assurance expert that generates functional test cases for websites. You take in a UI element and you generate a functional test case for usability.

Some UI elements have a link attached to them and other properties.
ONLY output in the following format: 
"Objective"~"Preconditions"~"Test Steps"~"Expected Result"
DO NOT output any other text. DO NOT output 'Here are the test cases...', your output should be like the example output below.

Example Input:
Link Element: Home with URL : https://bicol-u.edu.ph/
Link Element: Academics with URL : https://bicol-u.edu.ph/#
...

Example Output:
"Verify the functionality of the Link Element 'Home'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Home\'' '3. Verify if the webpage opens in a new tab/window.'"~"Webpage 'https://bicol-u.edu.ph/' should open in a new tab/window."
"Verify the functionality of the Link Element 'Academics'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Academics\'' '3. Verify if the link url changes to \'https://bicol-u.edu.ph/#\'' '4. Verify if a dropdown below \'Academics\' is visible'"~"A dropdown should show below 'Academics', but the webpage does not change"
...

Here is the UI element : {question}
"""

# Prompt Generator + LLM
# TODO : Experiment With Prompts
# TODO : Create Altered Version For Fine-Tuned Model Version (aka System Proper)

# Load Model Chain
def load_model_chain(template : str =  template, model_str : str = "llama3.1"):
    prompt = ChatPromptTemplate.from_template(template)
    model = OllamaLLM(model=model_str)
    chain = prompt | model
    return chain

# Create Test Case Data
def create_test_cases(data, model_str : str = "llama3.1" , template : str = template):
    
    # Load LLM Chain
    chain = load_model_chain(template, model_str)

    # Return Data
    return_data = []
    
    for item in data:
        return_data.append(chain.invoke({"question": str(item)}))
        # LLM Reset To Free Up Context
        chain = load_model_chain(template, model_str)

    return return_data


##### Version 1 (System Proper):

In [ ]:
# Langchain and Ollama
import langchain
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

# CONTEXT V1
template = """ Question:\n Generate test case for the following UI element: : {question} """
model_str = "llama3.1"

# Prompt Generator + LLM
# Load Model Chain
def load_model_chain(template : str =  template, model_str : str = model_str):
    # TODO : Plug In Fine Tuned Model
    prompt = ChatPromptTemplate.from_template(template)
    model = OllamaLLM(model=model_str)
    chain = prompt | model
    return chain

# Create Test Case Data
def create_test_cases(data):
    
    # Load LLM Chain
    chain = load_model_chain()

    # Return Data
    return_data = []
    
    for item in data:
        return_data.append(chain.invoke({"question": str(item)}))
        # LLM Reset To Free Up Context
        chain = load_model_chain()

    return return_data


##### Version 2: More Website Context

In [ ]:
# CONTEXT V2
# Aim to fix:
#   - url concern
#   - 

template = """
You are a quality assurance expert that generates functional test cases for websites. You take in a UI element and you generate a functional test case for usability.

Some UI elements have a link attached to them and other properties.
ONLY output in the following format: 
"Objective"~"Preconditions"~"Test Steps"~"Expected Result"
DO NOT output any other text. DO NOT output 'Here are the test cases...', your output should be like the example output below.

Example Input:
Link Element: Home with URL : https://bicol-u.edu.ph/
Link Element: Academics with URL : https://bicol-u.edu.ph/#
...

Example Output:
"Verify the functionality of the Link Element 'Home'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Home\'' '3. Verify if the webpage opens in a new tab/window.'"~"Webpage 'https://bicol-u.edu.ph/' should open in a new tab/window."
"Verify the functionality of the Link Element 'Academics'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Academics\'' '3. Verify if the link url changes to \'https://bicol-u.edu.ph/#\'' '4. Verify if a dropdown below \'Academics\' is visible'"~"A dropdown should show below 'Academics', but the webpage does not change"
...

Here is the UI element : {question}
The ui element is from the following website : {url}
"""

In [ ]:
"""
Generate a usability test case for the following UI element:

UI Element: Link
Element Name: 'All News'

Output should include:
1. Objectives for the link element
2. Preconditions for the link element
3. Test Steps for the link element
4. Expected Output for the link element

"""

In [1]:
# V2 Template
# TODO : Improve UI Element format for prompting
# TODO : Test against input texts more
# Used Techniques : Few shot prompting, specify context, and specify format.

template_2 = """
You are a quality assurance expert that generates functional test cases (usability) for websites. 

Generate a functional test case for usability for the following UI Element:
{ui_element}
The UI Element is from the following link:
{url}

Output should be in the form:
"Objective"~"Preconditions"~"Test Steps"~"Expected Output"

Example Input:
Link Element: Home with URL : https://bicol-u.edu.ph/ from url : https://bicol-u.edu.ph/
Link Element 'Battle of New Orleans' With URL https://en.wikipedia.org/wiki/Battle_of_New_Orleans from url : https://en.wikipedia.org/

Example Output:
"Verify the functionality of the Link Element 'Home'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Home\'' '3. Verify if the webpage opens in a new tab/window.'"~"Webpage 'https://bicol-u.edu.ph/' should open in a new tab/window."
"Verify the functionality of the Link Element 'Battle of New Orleans'"~"The user is on the webpage 'https://en.wikipedia.org/'"~"'1. User navigates to the webpage 'https://en.wikipedia.org/' '' '2. Click on Link Element 'Battle of New Orleans'' '3. Verify if the link url changes to 'https://en.wikipedia.org/wiki/Battle_of_New_Orleans' '4. Verify if the webpage opens in a new tab/window'"~"Webpage 'https://en.wikipedia.org/wiki/Battle_of_New_Orleans/' should open in a new tab/window."

"""

In [ ]:
# Fine Tuning Version

url = "placeholder"

# Load Model Chain
def load_model_chain(template : str =  template, model_str : str = "llama3.1"):
    prompt = ChatPromptTemplate.from_template(template)
    model = OllamaLLM(model=model_str)
    chain = prompt | model
    return chain

# Create Test Case Data
def create_test_cases(data, model_str : str = "llama3.1" , template : str = template, url : str = url):
    
    # Load LLM Chain
    chain = load_model_chain(template, model_str)

    # Return Data
    return_data = []
    
    i = 0
    total = len(data)
    for item in data:
        return_data.append(chain.invoke({"question": str(item), "url": url}))
        # LLM Reset To Free Up Context
        chain = load_model_chain(template, model_str)
        print(f"test case {i} out of {total} generated")

    return return_data